In [21]:
import pandas as pd
import dvc.api

In [8]:
from copy import deepcopy
from functools import reduce
import hashlib
from collections import OrderedDict
import json
def sortOD(od):
    res = OrderedDict()
    for k, v in sorted(od.items()):
        if isinstance(v, dict):
            res[k] = sortOD(v)
        else:
            res[k] = deepcopy(v)
    return res

In [34]:
df_metadata = pd.read_csv('../attribute_extraction_metadata_template/Initial Attribute Definition for First Release - UPDATED SHEET .csv')

In [35]:
df_metadata = df_metadata[['category', 'category_id', 'attribute_field', 'category_attributevalue', 'description', 'max_multi_select', 'usage', 'entry mode']]

In [36]:
metadata_category = set(df_metadata.category)
metadata_category_ids = set(df_metadata.category_id)

In [37]:
category2config = {}
for i in metadata_category:
    df_tmp = df_metadata[df_metadata.category == i]
    category2config[i] = sorted(df_tmp.to_dict('records'), key=lambda x: x['attribute_field'])
categoryid2config = {}
for i in metadata_category_ids:
    df_tmp = df_metadata[df_metadata.category_id == i]
    categoryid2config[i] = sorted(df_tmp.to_dict('records'), key=lambda x: x['attribute_field'])

In [38]:
df_tax = pd.read_json(dvc.api.get_url(
    'datasets/data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [39]:
tax2id = {}
for i in df_tax.to_dict('records'):
    if len(i['category_path']) > 0:
        tax2id[i['category_path']] = int(i['id'])

# sample file

In [6]:
df_samples = pd.read_csv('allstratified_sample_4806_query.csv').rename(columns={'top_query_classification_taxonomy': 'category_path'})

In [7]:
df_samples = df_samples[df_samples.category_path.apply(lambda x: any([i.startswith(x) for i in metadata_category]))]
df_samples = df_samples.sample(10, random_state=42)

In [8]:
df_samples

,label_ordering,query,sample_method,category_path
652,13434,tachos de basura,uniform,Home & Garden > Household Cleaning > Household...
2253,44644,robe hiver femme,head,Women's Clothing > Sweaters > Dresses
3839,79080,teck deck fingerboards,head,Toys & Hobbies
4119,84546,etagere salle de bain,head,Home Improvement > Bathroom Fixtures > Bathroo...
946,17674,cornici foto a muro,head,Home & Garden > Home Decor > Frames
2621,52701,dinossauros de brinquedo,head,Toys & Hobbies > Action & Toy Figures > Animal...
577,12139,pañuelos de algodon,uniform,Apparel Accessories > Men's Ties & Handkerchiefs
4180,85989,one piece mug,uniform,"Home & Garden > Kitchen,Dining & Bar > Drinkwa..."
650,13404,tenis para mujeres,head,Sports > Racquet Sports > Tennis > Tennis Shorts
4358,89949,school bags for kids,head,Luggage & Bags > Kids & Baby's Bags > School Bags


In [19]:
res = []
for i in df_samples[['query', 'sample_method', 'category_path']].to_dict('records'):
    i['text'] = f"Query: {i['query']}"
    attr_config = []
    for j in metadata_category:
        if j.startswith(i['category_path']):
            attr_config += deepcopy(category2config[j])
    i['attribute_config'] = attr_config
    task_json = json.dumps(sortOD(i)).encode('utf-8')
    task_id = hashlib.md5(task_json).hexdigest()
    i['task_id'] = f"query_attribution_md_{task_id}"
    i['preloading_result'] = None
    res.append(i)

In [27]:
pd.DataFrame(res).to_json('example_offshore_input_query_attr_extract_v2.json', lines=True, orient='records')

# demo file

In [40]:
df_samples = pd.read_csv(dvc.api.get_url(
    'modelling/notebooks/query_attr_extract_appen_label/appen_query_attribution_batch1.csv',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
)).rename(columns={'top_query_classification_taxonomy': 'category_path'})

In [41]:
df_samples[df_samples.category_path.apply(lambda x: x not in tax2id)]

,query,sample_method,label_ordering,category_path


In [42]:
[i for i in metadata_category if i not in tax2id]

[]

In [28]:
df_samples = df_samples[df_samples.category_path.apply(lambda x: any([i.startswith(x) for i in metadata_category]))]
df_samples = df_samples.sample(10, random_state=42)

In [43]:
df_samples

,query,sample_method,label_ordering,category_path
0,dog cat eater,uniform,0,Home & Garden > Pet Products > Cat Supplies > ...
1,fish holder tool,head,18,Sports > Fishing > Fishing Tools
2,caja para regalos,head,21,Home & Garden > Festive & Party Supplies > Wed...
3,inflow outflow pipe aquarium,head,22,Home & Garden > Household Merchandise > Lighte...
4,four pizza gaz,head,44,"Home & Garden > Kitchen,Dining & Bar > Bakewar..."
...,...,...,...,...
124995,phone case samsung a41,head,2487721,Cellphones & Telecommunications > Mobile Phone...
124996,mini maquinas de coser,head,2487731,"Home & Garden > Arts, Crafts & Sewing > Appare..."
124997,jewels for face,head,2487773,Jewelry & Accessories > Fine Jewelry > Loose D...
124998,album fotos 10x15,head,2487776,Home & Garden > Home Decor > Photo Albums


In [30]:
res = []
for i in df_samples[['query', 'sample_method', 'category_path']].to_dict('records'):
    i['text'] = f"Query: {i['query']}"
    attr_config = []
    for j in metadata_category:
        if j.startswith(i['category_path']):
            attr_config += deepcopy(category2config[j])
    i['attribute_config'] = attr_config
    task_json = json.dumps(sortOD(i)).encode('utf-8')
    task_id = hashlib.md5(task_json).hexdigest()
    i['task_id'] = f"query_attribution_md_{task_id}"
    i['preloading_result'] = None
    res.append(i)

In [33]:
pd.DataFrame(res).to_json('offshore_md/demo_query_attr_extract_md_02272023.json', lines=True, orient='records')